In [12]:
from sknn.mlp import Regressor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib
import sklearn
from sklearn_pandas import DataFrameMapper
matplotlib.style.use('ggplot')
%matplotlib inline

In [13]:
df = pd.read_csv('india_surroundings_1mar2016 _28_feb_2017.csv')
del df['IRIS ID'], df['Year'], df['Month'], df['Day'], df['Time UTC']
df = df.iloc[::-1]
df = pd.get_dummies(df)

In [14]:
train_test_ratio = .7
train_size = int(df.shape[0] * train_test_ratio)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]
df.head()

,Mag,Lat,Lon,Depth km,Timestamp,Region_AFGHANISTAN,Region_AFGHANISTAN-TAJIKISTAN BORD REG.,"Region_ANDAMAN ISLANDS, INDIA","Region_ANDAMAN ISLANDS, INDIA REGION",Region_EASTERN KASHMIR,...,Region_NORTHERN INDIA,Region_NORTHWESTERN KASHMIR,Region_PAKISTAN,"Region_QINGHAI, CHINA",Region_SOUTHEASTERN AFGHANISTAN,Region_SOUTHERN INDIA,"Region_SOUTHERN XINJIANG, CHINA",Region_TAJIKISTAN,Region_WESTERN XIZANG-INDIA BORDER REG.,Region_XIZANG
279,4.6,29.75,69.93,10.0,1456900511,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
278,4.0,35.53,70.24,89.5,1457132148,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277,4.2,36.49,71.10,101.3,1457232360,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
276,4.6,20.98,95.02,101.0,1457320438,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275,4.5,22.76,93.94,50.0,1457353381,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
mapper = DataFrameMapper([
       # (['Region'], None),
        (['Mag'],sklearn.preprocessing.StandardScaler()),
        (['Depth km'],sklearn.preprocessing.StandardScaler()),
        (['Lat'],sklearn.preprocessing.StandardScaler()),
        (['Lon'],sklearn.preprocessing.StandardScaler()),
        (['Timestamp'], sklearn.preprocessing.StandardScaler()) 
    ],default = None)
train_data = mapper.fit_transform(train_data)
#train_data[['Mag','Lat','Lon','Timestamp','Depth km']] = train_data[['Mag','Lat','Lon','Timestamp','Depth km']].astype(float)
test_data = mapper.transform(test_data)
#test_data[['Mag','Lat','Lon','Timestamp','Depth km']] = test_data[['Mag','Lat','Lon','Timestamp','Depth km']].astype(float)


/home/nikhil/.conda/envs/sarcoma/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [22]:
past = 14  # number of past earthquakes to consider 
def getXandY(data):
    X, Y = [], []
    for i in range(data.shape[0]-past):
        X += [data[i:i+past].flatten()]
        Y += [data[i+past]]
    return X, Y
train_X, train_Y = getXandY(train_data)
test_X, test_Y = getXandY(test_data)

In [29]:
print(np.shape(train_X), np.shape(train_Y))
print(np.shape(test_X), np.shape(test_Y))

(182, 448) (182, 32)
(70, 448) (70, 32)


(2, 32)